# Import Demand

In [840]:
import math

map_file_name = 'data/disciplinas/dia_hora_instituto_ponto_linha_funcionarios_18h.json'

sentido = 'ida'
acao = 'desembarque'

hora = 1080
def intervalo_horario_am_pm(horario):
    if isinstance(horario, float) and math.isnan(horario):
        return False
    return '5:30:00 PM' <= horario <= '7:30:00 PM'
def intervalo_horario_inicio(horario_inicio, horario_fim):
    return '15:30' <= horario_inicio <= '17:30'
def intervalo_horario_fim(horario_inicio, horario_fim):
    return '17:30' <= horario_fim <= '19:00'
intervalo_horario = intervalo_horario_inicio if sentido == 'ida' else intervalo_horario_fim

def get_dados_vinculo(aulas_alunos, funcionarios_intervalo_horario):
    return funcionarios_intervalo_horario
def grad_ou_pos(vinculo):
    return vinculo == 'Aluno de graduação' or vinculo == 'Aluno de pós-graduação'
def funcionario(vinculo):
    return vinculo == 'Funcionário'
filtra_vinculo = funcionario

dias = ['seg', 'ter', 'qua', 'qui', 'sex', 'sab', 'dom']

# Aulas MATRUSP

##### Read jsons

In [841]:
import os
import json

In [842]:
files = []
dir_path = 'data/disciplinas/json/'
for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        files.append(path)

In [843]:
len(files)

5205

In [844]:
disciplinas = []
for file in files:
    with open(dir_path + file, 'r+') as f:
        content = json.loads(f.read())
        disciplinas.append(content)

In [845]:
disciplinas[1000]

{'unidade': 'Escola Politécnica',
 'departamento': 'Eng Telecomunicações e Controle',
 'campus': 'São Paulo',
 'codigo': 'PTC3502',
 'nome': 'Estágio Supervisionado',
 'creditos_aula': 1,
 'creditos_trabalho': 6,
 'objetivos': 'Realização de um estágio relacionado ao futuro exercício profissional, com a supervisão de um professor do curso.',
 'programa_resumido': 'Estágio em empresa, universidade ou instituição de pesquisa.',
 'turmas': [{'codigo': '2022250',
   'inicio': '15/08/2022',
   'fim': '21/12/2022',
   'tipo': 'Teórica',
   'horario': [{'dia': 'sab',
     'inicio': '09:20',
     'fim': '10:10',
     'professores': ['Cristiano Magalhaes Panazio']}],
   'vagas': {'Obrigatória': {'vagas': 35,
     'inscritos': 8,
     'pendentes': 0,
     'matriculados': 8,
     'grupos': {'EP - Engenharia Elétrica - EC3 (Telecomunicações)': {'vagas': 35,
       'inscritos': 6,
       'pendentes': 0,
       'matriculados': 6}}},
    'Extracurricular': {'vagas': 2,
     'inscritos': 0,
     'pend

##### Filtrar Institutos CUASO

In [846]:
fora_cuaso = ['Escola de Artes, Ciências e Humanidades', 'Escola de Enfermagem', 'Faculdade de Direito', 'Faculdade de Medicina',
              'Faculdade de Saúde Pública', 'Faculdade de Saúde Pública e Faculdade de Ciências Farmacêuticas', 'Escola de Enfermagem de Ribeirão Preto',
              'Museu de Zoologia']
def is_disciplina_cuaso(disciplina, fora_cuaso):
    if disciplina['campus'] != 'São Paulo':
        return False
    if disciplina['unidade'] in fora_cuaso:
        return False
    return True

In [847]:
disciplinas_cuaso = []
for disciplina in disciplinas:
    if is_disciplina_cuaso(disciplina, fora_cuaso):
        disciplinas_cuaso.append(disciplina)
len(disciplinas_cuaso)

2352

In [848]:
unidades_matrusp = {}
for disciplina in disciplinas_cuaso:
    unidades_matrusp[disciplina['unidade']] = 1
unidades_matrusp

{'Instituto Oceanográfico': 1,
 'Escola Politécnica': 1,
 'Escola de Comunicações e Artes': 1,
 'Instituto de Matemática e Estatística': 1,
 'Faculdade de Filosofia, Letras e Ciências Humanas': 1,
 'Instituto de Química': 1,
 'Instituto de Ciências Biomédicas': 1,
 'Instituto de Geociências': 1,
 'Instituto de Biociências': 1,
 'Instituto de Psicologia': 1,
 'Faculdade de Medicina Veterinária e Zootecnia': 1,
 'Faculdade de Ciências Farmacêuticas': 1,
 'Escola de Educação Física e Esporte': 1,
 'Faculdade de Educação': 1,
 'Instituto de Física': 1,
 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 1,
 'Faculdade de Odontologia': 1,
 'Faculdade de Arquitetura e Urbanismo': 1,
 'Instituto de Relações Internacionais': 1,
 'Faculdade de Economia, Administração, Contabilidade e Atuária': 1,
 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências': 1,
 'Instituto de Estudos Brasileiros': 1,
 'Museu de Arqueologia e Etnologia': 1

##### Gerar Lista de Aulas

In [849]:
aulas = []
for i, disciplina in enumerate(disciplinas_cuaso):
    for turma in disciplina['turmas']:
        total_matriculados = 0
        for k, vaga in turma['vagas'].items():
            total_matriculados += vaga['matriculados']
            
        if turma['horario'] != None:
            for horarios in turma['horario']:
                aula = {}
                aula['indice_disc'] = i
                aula['codigo'] = disciplina['codigo']
                aula['codigo_turma'] = turma['codigo']
                aula['unidade'] = disciplina['unidade']
                aula['dia'] = horarios['dia']
                aula['horario_inicio'] = horarios['inicio']
                aula['horario_fim'] = horarios['fim']
                aula['total'] = total_matriculados
                aulas.append(aula)

##### Filtrar Aulas por Intervalo Horário

In [850]:
aulas_intervalo_horario = []
for aula in aulas:
    if intervalo_horario(aula['horario_inicio'], aula['horario_fim']):
        aulas_intervalo_horario.append(aula)

# Pesquisa Parte 1

##### Ler Pesquisa

In [851]:
import pandas as pd
import datetime as dt

In [852]:
pesquisa_perguntas = pd.read_csv('./data/disciplinas/forms/resposta_pesq.csv', delimiter=';')
pesquisa_perguntas['timestamp'] = list(map(lambda x: dt.datetime.strptime(x, '%m/%d/%Y %H:%M:%S'), pesquisa_perguntas['Carimbo de data/hora']))
pesquisa_perguntas = pesquisa_perguntas[pesquisa_perguntas['timestamp'] >= dt.datetime(2022, 10, 7, 0, 0, 0)]
len(pesquisa_perguntas)

4395

In [853]:
pesquisa = pd.DataFrame()
pesquisa['vinculo'] = pesquisa_perguntas['Qual o seu vínculo com a USP?']
pesquisa['instituto'] = pesquisa_perguntas['A qual instituto você está associado?']


# ida
pesquisa['utiliza_circular_ida'] = pesquisa_perguntas['Quantas vezes por semana você utiliza o circular para entrar na USP?'] >= 2
pesquisa['linhas_ida'] = pesquisa_perguntas['Qual das linhas abaixo você utiliza neste trajeto?']
pesquisa['ponto_embarque_ida'] = pesquisa_perguntas['Por qual ponto você normalmente embarca?']
pesquisa['ponto_desembarque_ida'] = pesquisa_perguntas['Em qual ponto de ônibus você normalmente desembarca?']
pesquisa['horario_ida'] = pesquisa_perguntas['Que horas você normalmente chega na USP?']


# volta
pesquisa['utiliza_circular_volta'] = pesquisa_perguntas['Quantas vezes por semana você utiliza o circular para sair da USP?'] >= 2
pesquisa['linhas_volta'] = pesquisa_perguntas['Qual das linhas abaixo você utiliza neste trajeto?.1']
pesquisa['ponto_embarque_volta'] = pesquisa_perguntas['Em qual o ponto de ônibus você embarca para sair da USP?']
pesquisa['ponto_desembarque_volta'] = pesquisa_perguntas['Por qual ponto você normalmente desembarca?']
pesquisa['horario_volta'] = pesquisa_perguntas['Que horas você normalmente chega no ponto de ônibus para sair da USP?']

pesquisa.head()

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
136,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8022",P3,FAU II,7:54:00 AM,True,"8012, 8022",FAU II,Portão da Vila Indiana,7:10:00 PM
137,Aluno de graduação,Faculadae de Arquitetura e Urbanismo (FAU),True,"8012, 8032",Estação Butantã,FEA,1:10:00 PM,True,"8012, 8022",FAU II,Estação Butantã,10:30:00 PM
138,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8032",Estação Butantã,FEA,7:15:00 PM,True,8022,Rua do Matão 0,Estação Butantã,10:40:00 PM
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,8012,PTREM,Geografia/História,7:25:00 PM,True,8012,Geografia/História,PTREM,9:40:00 PM
140,Aluno de graduação,Instituto de Relações internacionais (IRI),False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN


In [854]:
pesquisa['instituto'].unique()

array(['Instituto de Matemática e Estatística (IME)',
       'Faculadae de Arquitetura e Urbanismo (FAU)',
       'Faculdade de Filosofia, Letras e Ciências Humanas (FFLCH)',
       'Instituto de Relações internacionais (IRI)',
       'Faculdade de Ciências Farmacêuticas (FCF)',
       'Escola Politécnica (Poli)',
       'Faculdade de Economia, Administração e Contabilidade (FEA)',
       'Instituto de Química (IQ)',
       'Escola de Artes, Ciências e Humanidades (EACH)',
       'Instituto de Física (IF)', 'Faculdade de Medicina (FM)',
       'Instituto Oceanográfico (IO)',
       'Escola de Comunicações e Artes (ECA)',
       'Instituto de Pesquisas Energéticas e Nucleares (IPEN)',
       'Instituto de Biociências (IB)', 'Faculdade de Educação (FE)',
       'Instituto de Ciências Biomédicas (ICB)',
       'Faculdade de Odontologia (FO)',
       'Escola de Educação Física e Esporte (EEFE)', 'PRG',
       'Instituo de Astronomia, Geofísica e Ciências Atmosféricas (IAG)',
       'Instit

##### Filtrar Vínculo

In [855]:
pesquisa[['vinculo']].value_counts()

vinculo                                                        
Aluno de graduação                                                 2880
Aluno de pós-graduação                                              752
Funcionário                                                         644
Docente                                                              36
docente                                                              22
Professor                                                            21
professor                                                             8
Ex-aluno                                                              5
Professor                                                             5
Docente                                                               4
Professor titular                                                     1
aprimoramento                                                         1
Pós-doutorando                                                        1


In [856]:
pesquisa[['vinculo', 'utiliza_circular_' + sentido]].value_counts()

vinculo                                                          utiliza_circular_ida
Aluno de graduação                                               True                    2532
Aluno de pós-graduação                                           True                     575
Funcionário                                                      True                     369
Aluno de graduação                                               False                    348
Funcionário                                                      False                    275
Aluno de pós-graduação                                           False                    177
Docente                                                          True                      28
docente                                                          False                     17
Professor                                                        True                      11
                                                                 Fal

In [857]:
pesquisa_vinculo = pesquisa[list(map(filtra_vinculo, pesquisa['vinculo']))]
pesquisa_vinculo[['vinculo']].value_counts()

vinculo    
Funcionário    644
dtype: int64

##### Normaliza instituto com unidade do matrusp ou anuário

In [858]:
def retira_sigla(x):
    index = x.find('(')
    
    if index < 0:
        return x
    
    return x[:index - 1]

def filtra_instituto_pesq(x):
    x = retira_sigla(x)
    
    if x in ['ECA e MAC USP', 'Escola de Artes, Ciências e Humanidades']: 
        return 'Escola de Comunicações e Artes'
    elif x == 'FFLCH':
        return 'Faculdade de Filosofia, Letras e Ciências Humanas'
    elif x in ['IPEN', 'Instituto de Pesquisas Energéticas e Necleares', 'IPEN ', 'ipen', 'Instituto de pesquisas energéticas e nucleares', 'Instituto de Pesquisa Energéticas e Nucleares', 'Instituto de Pesquisas Energéticas e Nucleares - IPEN', 'Instituto de pesquisas energéticas e nucleares ', 'Instituto de pesquisas energéticas e nucleares -IPEN', 'Instituo de Pesquisas Energéticas e Nucleares']:
        return 'Instituto de Pesquisas Energéticas e Nucleares'
    elif x in ['MAC', 'MAC USP']:
        return 'Museu de Arte Contemporânea'
    elif x == 'Faculadae de Arquitetura e Urbanismo':
        return 'Faculdade de Arquitetura e Urbanismo'
    elif x in ['IAG ', 'IAG', 'Instituo de Astronomia, Geofísica e Ciências Atmosféricas']:
        return 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas'
    elif x == 'Faculdade de Economia, Administração e Contabilidade':
        return 'Faculdade de Economia, Administração, Contabilidade e Atuária'
    elif x in ['Escola de Aplicação da Faculdade de Educação (EA-FEUSP)']:
        return 'Escola de Educação Física e Esporte'
    elif x == 'Instituto de Relações internacionais':
        return 'Instituto de Relações Internacionais'
    elif x in ['Museu de arqueologia e etnologia ']:
        return 'Museu de Arqueologia e Etnologia'
    elif x in ['Instituto de Energia e Ambientes']:
        return 'Instituto de Energia e Ambiente'
    elif x in ['Hospital universitário da usp', 'Hospital universitário ', 'Hospital HU', 'Hosp. Universitario', 'HU-USP', 'Hospital Universitário da USP ', 'HOSPITAL UNIVERSITÁRIO', 'HOSPITAL UNIVERSITÁRIO - HU', 'Hospital Universitário da USP', 'HU USP', 'hospital universitário', 'HU', 'Hospital Universitário HU', 'Hu', 'HOSPITAL UNIVERSITARIO', 'Hospital universitário', 'Hospital  Universitário ', 'hu', 'HU ', 'Hospital Universitário ']:
        return 'Hospital Universitário'
    elif x in ['SAS', 'sas', 'Superintendência de Comunicação Social', 'Prip', 'edusp', 'pusp c', 'PUSP-C','pusp-c', 'Prédio da Reitoria', 'Prédio da Administração Central-SEF', 'Prefeitura do Campus USP da Capital', 'SUB REITORIA', 'Reitoria.', 'Prip usp', 'Pró-Reitoria de Inclusão e Pertencimento - PRIP', 'pro reitoria', 'Pro reitoria de cultura e extensão ', 'PRIP/Reitoria', 'Edusp', 'PUSP C', 'PUS-C', 'Pusp', 'Prefeitura do Campos', 'Prefeitura ', 'Reitoria da USP', 'Editora da Universidade de São Paulo - Edusp', 'Pusp-c', 'PRG', 'Prg ', 'Pró-Reitoria de Graduação', 'Pró-Reitoria de graduação', 'Pró-reitoria de graduacao', 'Reitoria', 'Reitoria usp', 'REITORIA', 'Pró-Reitoria de Inclusão e Pertencimento - PRIP-USP', 'Pró-Reitoria de Inclusão e Pertencimento', 'Reitoria da Universidade de São Paulo', 'Reitoria - Administração Central', 'Reitoria ', 'rusp', 'RUSP', 'Editora da USP']:
        return 'Reitoria Administração Central'
    elif x in ['Museu de Zoologia', 'Museu de Zoologia da USP']:
        return 'Faculdade de Medicina Veterinária e Zootecnia'
    elif x in ['Superintendencia de Tecnologia da Informação', 'superintendencia do espaço fisico', 'Superintendência de Segurança - SPPU', 'SUperintendencia de Tecnologia e INformação', 'STI - CeTI-SP']:
        return 'Superintendência da Tecnologia da Informação'
    elif x in ['Cepeusp ', 'Cepeusp', 'Centro de Práticas Esportivas da USP - CEPEUSP', 'CEPEUSP']:
        return 'Centro de Práticas Esportivas'
    elif x in ['Curso de Ciências Moleculares', 'Prceu ', 'PRCEU', 'Clinica Odontológica da SAU no CRUSP', 'PRIP', 'STI', 'Superintendência de Tecnologia da Informação - STI', 'CEPEUSP ', 'SEF', 'Coseas', 'Superintendência do Espaço Físico', 'BBM-USP', 'CeTISP', 'ABCD', 'Restaurante universitario central', 'Restaurante Central ', 'Aucani', 'Centro de Saúde Escola Samuel Barsley Pessoa ', 'Osusp', 'SCS', 'Museu Paulista', 'Museu de Arqueologia e Etnologia ', 'CSCRH-RP-01', 'Instituto de Medicina Tropical de São Paulo', 'Rádio USP ', 'Perceu', 'USP Centro de Tecnologia de Informação - CETISP']:
        None
    else:
        return x
    
pesquisa_vinculo2 = pesquisa_vinculo.copy()
pesquisa_vinculo2['instituto'] = list(map(filtra_instituto_pesq, pesquisa_vinculo['instituto']))

In [859]:
# Filtra os Nones

pesquisa_vinculo = pesquisa_vinculo2
pesquisa_vinculo = pesquisa_vinculo[pesquisa_vinculo['instituto'].notna()]
pesquisa_vinculo['instituto'].unique()

array(['Instituto de Matemática e Estatística', 'Instituto de Física',
       'Hospital Universitário', 'Reitoria Administração Central',
       'Faculdade de Medicina', 'Faculdade de Educação',
       'Faculdade de Medicina Veterinária e Zootecnia',
       'Faculdade de Odontologia', 'Escola de Comunicações e Artes',
       'Faculdade de Ciências Farmacêuticas',
       'Faculdade de Filosofia, Letras e Ciências Humanas',
       'Escola Politécnica', 'Instituto de Geociências',
       'Instituto de Ciências Biomédicas', 'Instituto de Química',
       'Escola de Educação Física e Esporte',
       'Museu de Arte Contemporânea',
       'Faculdade de Economia, Administração, Contabilidade e Atuária',
       'Instituto Oceanográfico', 'Instituto de Energia e Ambiente',
       'Instituto de Biociências', 'Escola de Enfermagem',
       'Hospital Universitario', 'Faculdade de Arquitetura e Urbanismo',
       'Instituto de Astronomia, Geofísica e Ciências Atmosféricas',
       'Instituto de Rel

##### Mapeia Institutos Fora CUASO

In [860]:
def mapeia_cursos_de_fora(x):
    if x in ['Faculdade de Medicina', 'Escola de Enfermagem', 'Faculdade de Saúde Pública']:
        return 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências'
    elif x == 'Faculdade de Direito':
        return 'Faculdade de Filosofia, Letras e Ciências Humanas'
    elif x == 'MAC':
        return 'Museu de Arte Contemporânea'
    else:
        return x
    
pesquisa_vinculo2 = pesquisa_vinculo.copy()
pesquisa_vinculo2['instituto'] = list(map(mapeia_cursos_de_fora, pesquisa_vinculo['instituto']))
pesquisa_vinculo = pesquisa_vinculo2

In [861]:
pesquisa_vinculo['instituto'].unique()

array(['Instituto de Matemática e Estatística', 'Instituto de Física',
       'Hospital Universitário', 'Reitoria Administração Central',
       'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências',
       'Faculdade de Educação',
       'Faculdade de Medicina Veterinária e Zootecnia',
       'Faculdade de Odontologia', 'Escola de Comunicações e Artes',
       'Faculdade de Ciências Farmacêuticas',
       'Faculdade de Filosofia, Letras e Ciências Humanas',
       'Escola Politécnica', 'Instituto de Geociências',
       'Instituto de Ciências Biomédicas', 'Instituto de Química',
       'Escola de Educação Física e Esporte',
       'Museu de Arte Contemporânea',
       'Faculdade de Economia, Administração, Contabilidade e Atuária',
       'Instituto Oceanográfico', 'Instituto de Energia e Ambiente',
       'Instituto de Biociências', 'Hospital Universitario',
       'Faculdade de Arquitetura e Urbanismo',
       'Instituto de Astron

##### Filtrar respostas pesquisa pelo Intervalo Horário

In [862]:
pesquisa_vinculo.head()

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
249,Funcionário,Instituto de Matemática e Estatística,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
282,Funcionário,Instituto de Física,False,"8012, 8022, 8032",P1,Poli Biênio,7:30:00 AM,True,8022,Poli Biênio,Estação Butantã,5:00:00 PM
523,Funcionário,Hospital Universitário,True,"8012, 8032",Estação Butantã,Hospital Universitário,7:00:00 AM,True,8032,Hospital Universitário,Estação Butantã,7:05:00 PM
601,Funcionário,Reitoria Administração Central,True,"8012, 8032",Estação Butantã,Reitoria/Bancos,9:05:00 AM,True,"8012, 8032",Psicologia II,Estação Butantã,6:10:00 PM
844,Funcionário,"Faculdade de Medicina, Instituto de Ciências B...",True,8012,Estação Butantã,P3,9:00:00 AM,True,8022,P3,Estação Butantã,6:00:00 PM


In [863]:
pesquisa_vinculo['no_horario'] = list(map(intervalo_horario_am_pm, pesquisa_vinculo['horario_' + sentido]))
pesquisa_vinculo_intervalo = pesquisa_vinculo[pesquisa_vinculo['no_horario']]
pesquisa_vinculo_intervalo

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta,no_horario
282,Funcionário,Instituto de Física,False,"8012, 8022, 8032",P1,Poli Biênio,7:30:00 AM,True,8022,Poli Biênio,Estação Butantã,5:00:00 PM,True
523,Funcionário,Hospital Universitário,True,"8012, 8032",Estação Butantã,Hospital Universitário,7:00:00 AM,True,8032,Hospital Universitário,Estação Butantã,7:05:00 PM,True
972,Funcionário,Faculdade de Educação,True,8022,Portão da Vila Indiana,Avenida da Universidade,7:05:00 AM,False,"8012, 8022",Avenida da Universidade,Estação Butantã,2:15:00 PM,True
1381,Funcionário,Faculdade de Ciências Farmacêuticas,True,8022,P3,Portão da Vila Indiana,7:00:00 AM,True,8022,Farmácia e Química,P3,4:15:00 PM,True
1424,Funcionário,"Faculdade de Filosofia, Letras e Ciências Humanas",True,"8012, 8022, 8032",Portaria São Remo,Geografia/História,7:00:00 AM,True,8012,Letras,Portaria São Remo,5:00:00 AM,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4201,Funcionário,Reitoria Administração Central,True,8022,P3,Cultura Japonesa,7:20:00 AM,True,8012,CRUSP (Praça da Reitoria),P3,5:30:00 PM,True
4387,Funcionário,Instituto de Química,True,8022,PTREM,PTREM,6:40:00 AM,True,8022,Estação Butantã,PTREM,4:40:00 PM,True
4391,Funcionário,Reitoria Administração Central,True,"8012, 8032",P3,Centro De Difusão Internacional,7:30:00 AM,True,8012,Centro De Difusão Internacional,Estação Butantã,5:00:00 PM,True
4449,Funcionário,Faculdade de Odontologia,True,8012,Estação Butantã,Odontologia,6:25:00 AM,True,8022,Ipen,Estação Butantã,3:40:00 PM,True


# Funcionários

##### Ler os dados do anuário

In [864]:
with open('data/funcionarios_anuario.json', 'r+') as f:
    str_funcionarios_anuario = f.read()
    funcionarios_anuario = json.loads(str_funcionarios_anuario)
funcionarios_anuario[0]

{'unidade': 'Faculdade de Arquitetura e Urbanismo', 'funcionarios': 129}

##### Filtrar Horário baseado na pesquisa

In [865]:
no_horario_por_instituto = pesquisa_vinculo[['instituto', 'no_horario']].groupby('instituto').agg('mean').reset_index()
no_horario_por_instituto[no_horario_por_instituto['instituto'] == 'Instituto de Matemática e Estatística']

,instituto,no_horario
23,Instituto de Matemática e Estatística,0.25


In [866]:
funcionarios_intervalo_horario = []
for funcionario_anuario in funcionarios_anuario:
    unidade = funcionario_anuario['unidade']
    porc_no_horario = no_horario_por_instituto[no_horario_por_instituto['instituto'] == unidade]['no_horario'].iloc[0]
    for dia in dias:
        funcionarios_intervalo_horario.append({
            'unidade': unidade,
            'dia': dia,
            'total': funcionario_anuario['funcionarios'] * porc_no_horario,
        })
funcionarios_intervalo_horario[0]

{'unidade': 'Faculdade de Arquitetura e Urbanismo',
 'dia': 'seg',
 'total': 9.923076923076923}

# Pesquisa Parte 2

In [ ]:
pesquisa_vinculo_intervalo['ponto_embarque_ida'].unique()

##### Filtrar Butantã

In [867]:
pesquisa_vinculo_intervalo_butanta = pesquisa_vinculo_intervalo[pesquisa_vinculo_intervalo['utiliza_circular_' + sentido]]
def is_butanta(x):
    return x == 'Estação Butantã' or x == 'PTREM'
pesquisa_vinculo_intervalo_butanta = pesquisa_vinculo_intervalo_butanta[list(map(is_butanta, pesquisa_vinculo_intervalo_butanta['ponto_' + acao + '_' + sentido]))]
pesquisa_vinculo_intervalo_butanta

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta,no_horario
523,Funcionário,Hospital Universitário,True,"8012, 8032",Estação Butantã,Hospital Universitário,7:00:00 AM,True,8032,Hospital Universitário,Estação Butantã,7:05:00 PM,True
1623,Funcionário,"Faculdade de Medicina, Instituto de Ciências B...",True,8012,Estação Butantã,Hospital Universitário,6:35:00 AM,True,8012,Hospital Universitário,Estação Butantã,7:00:00 PM,True
1638,Funcionário,Instituto de Ciências Biomédicas,True,"8012, 8022",Estação Butantã,Biomédicas,6:30:00 AM,True,8022,Biomédicas,Estação Butantã,2:45:00 PM,True
1652,Funcionário,Reitoria Administração Central,True,8022,PTREM,IPT,6:15:00 AM,True,8012,PTREM,PTREM,4:00:00 PM,True
1659,Funcionário,Instituto de Química,True,8022,Estação Butantã,Farmácia e Química,7:30:00 AM,True,8022,Farmácia e Química,Estação Butantã,10:00:00 PM,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4171,Funcionário,Reitoria Administração Central,True,"8012, 8032",Estação Butantã,CRUSP (Praça da Reitoria),7:30:00 AM,True,8032,Estação Butantã,Estação Butantã,5:00:00 PM,True
4173,Funcionário,"Faculdade de Filosofia, Letras e Ciências Humanas",True,"8012, 8022, 8032",Estação Butantã,Reitoria/Bancos,7:20:00 AM,True,8022,rua do matao,Estação Butantã,5:30:00 PM,True
4175,Funcionário,Escola Politécnica,True,"8012, 8022, 8032",Estação Butantã,Poli-Eletrotécnia,6:50:00 AM,True,"8012, 8032",Poli-Eletrotécnia,Estação Butantã,4:00:00 PM,True
4387,Funcionário,Instituto de Química,True,8022,PTREM,PTREM,6:40:00 AM,True,8022,Estação Butantã,PTREM,4:40:00 PM,True


##### Mapeia Pontos Pesquisa para ficar igual ao da API

In [868]:
import requests
pontos = requests.get('https://uspdigital.usp.br/mobile/servicos/mapa/locais?campus=cidade-universitaria&categorias=transportes')
pontos = json.loads(pontos.text)
pontos[10]

{'categoria': 'transportes',
 'campus': 'cidade-universitaria',
 'id': '1811',
 'titulo': 'Ponto de ônibus ECA',
 'latitude': '-23.557806338669685',
 'longitude': '-46.72689100000002',
 'endereco': 'Av. Prof. Lúcio Martins Rodrigues',
 'imagem': 'http://www.usp.br/mapas/wp-content/uploads/10042014giroculturalfotomarcossantos008.jpg',
 'url': '',
 'descricao': 'Atende as linhas  701U-10, 702U-10, 7725-10, 8012-10'}

In [869]:
nomes_pontos_req = []
for ponto in pontos:
    if ponto['campus'] == 'cidade-universitaria':
        nomes_pontos_req.append(ponto['titulo'])
 
nomes_pontos_pesq = []
for ponto in pesquisa_vinculo_intervalo['ponto_' + acao + '_' + sentido].unique():
    nomes_pontos_pesq.append(ponto)

print(nomes_pontos_req)
print(nomes_pontos_pesq)

['Ponto de ônibus metrô Butantã', 'Ponto de ônibus Afrânio Peixoto', 'Ponto de ônibus Poli Metalúrgica', 'Terminal de ônibus USP', 'Ponto de ônibus Poli Mecânica', 'Ponto de ônibus portaria II', 'Ponto de ônibus Hidráulica', 'Ponto de Psicologia I', 'Ponto de ônibus Barracões', 'Ponto de ônibus ECA', 'Ponto de ônibus Praça do Relógio', 'Ponto de ônibus - Praça do Relógio', 'Ponto de ônibus Psicologia II', 'Ponto de ônibus - Acesso CPTM II', 'Ponto de ônibus - Acesso CPTM I', 'Ponto de ônibus Escola de Educação Física II', 'Ponto de ônibus Escola de Educação Física I', 'Ponto de ônibus Academia de Polícia', 'Ponto de ônibus Paços das Artes', 'Ponto de ônibus Educação', 'Ponto de ônibus CRUSP', 'Ponto de ônibus Cultura Japonesa', 'Ponto de ônibus Biblioteca Brasiliana', 'Ponto de ônibus Letras', 'Ponto de ônibus Geociência', 'Ponto de ônibus Reitoria/Bancos', 'Ponto de ônibus FEA', 'Ponto de ônibus FAU II', 'Ponto de ônibus Poli Biênio', 'Ponto de ônibus Poli Eletrotécnica', 'Ponto de ôn

In [870]:
def iguais_com_prefixo(x):
    aux_nomes_pontos_pesq = ['Ponto de ônibus ' + x for x in nomes_pontos_pesq]
    if 'Ponto de ônibus ' + x in [item for item in aux_nomes_pontos_pesq if item in list(set(nomes_pontos_req) & set(aux_nomes_pontos_pesq))]:
        return 'Ponto de ônibus ' + x
    else:
        return x
 
def pesquisa_to_pontos_req(x):
    if x in ["Geografia/História", "História e Geografia", "Químicas"]:
        return "Ponto e ônibus História e Geografia"
    elif x  == "Poli Metalurgia":
        return "Ponto de ônibus Poli Metalúrgica"
    elif x == "P3":
        return "Ponto de ônibus Portaria III"
    elif x == "Farmácia e Química":
        return "Ponto de ônibus Biblioteca Farmácia e Química"
    elif x in ["CRUSP (Praça da Reitoria)", "R. Prof. Mello Moraes, 1473 (localizado entre a CRUSP e a Raia, sentindo Metrô Butantã)", "R. Prof. Mello Moraes, 900 (localizado entre a CRUSP e a Raia, sentindo P3)"]:
        return "Ponto de ônibus CRUSP"
    elif x == "Biomédicas":
        return "Ponto de ônibus Biomédicas III"
    elif x in ["Prefeitura/Física", "Ponto de ônibus Ponto Clube dos funcionários"]:
        return "Ponto Clube dos funcionários"
    elif x == "Educação Física":
        return "Ponto de ônibus Escola de Educação Física I"
    elif x in ["Estação Butantã", "Terminal Cidade Universitária"]:
        return "Ponto de ônibus metrô Butantã"
    elif x == "Poli-Eletrotécnia":
        return "Ponto de ônibus Poli Eletrotécnica"
    elif x == "Geociências":
        return "Ponto de ônibus Geociências I"
    elif x in ["icb ", "Biociências", "ICB 1 (Av. Prof. Lineu Prestes, 1524)"]:
        return "Ponto de ônibus  Parada Matemática"
    elif x in ["FE", "Avenida da Universidade", "Faculdade de Educação ", "Educação / CEPEUSP", " FE", "Prédio da FE", "Educação ", "Educação", "Faculdade de Educação"]:
        return "Ponto de ônibus Educação"
    elif x in ["Rua do matão frente da FAU", "Instituto Oceanografico", "Oceanográfico, em frent ao da FAU II", "Rua do Matão, 0 (em frente a FAU)", "Estacionamento FAU", "R. do Matão, frente à FAU", "R. do Matão 878", "R. do Matão, 0", "Oceanográfico", "Oceanografia", "Rua do Matao ", "Matão, Oceanografico", "Rua do Matão, 0 - FAU", "IO USP", "IO", "Rua do matao, 0", "Rua do Matão 0 (FAU)", "Rua do Matão (na frente da FAU)", "Intuito Oceanografico", "R. do Matão", "Oceanografia ", "Oceanógrafico/Matão", "Rua do Matão, 0", "R. Do matão ",  "Rua do Matão", "Ponto em frente à FAU na Rua do Matão"]:
        return "Ponto de ônibus FAU I"
    elif x in ["depois da biblioteca da química ", "Rua do Lago", "Lineu Prestes entre a biblioteca da quimica e o icb", "Ponto do Lago", "Av. Prof. Lineu Prestes, 0"]:
        return "Ponto de ônibus rua do Lago"
    elif x == "Psicologia I":
        return "Ponto de ônibus Psicologia II"
    elif x in ["Cepam",  "Biomédicas, Biologia e Quimica"]:
        return "Ponto de ônibus CEPAM"
    elif x in ["Ipen", "FO ou Ipen", "Marinha/ Odontologia"]:
        return "Ponto de ônibus IPEN"
    elif x in ["Acesso Vila Indiana", "Portão da Vila Indiana"]:
        return "Ponto de ônibus acesso Vl. Indiana"
    elif x == "PTREM":
        return "Ponto de ônibus - Acesso CPTM I"
    elif x == "IPT":
        return "Ponto de ônibus Cocesp I"
    elif x in ["Centro De Difusão Internacional", "Poli Hidráulica"]:
        return "Ponto de ônibus Hidráulica"
    elif x == "Avenida Professor Luciano Gualberto (depois da Parada I.P.T, no 8022)":
        return "Ponto de ônibus Poli Eletrotécnica"
    elif x in ["P1", "Instituto Butantan ", "Academia de polícia"]:
        return "Ponto de ônibus Academia de Polícia"
    elif x == "P2":
        return "Ponto de ônibus portaria II"
    elif x in ["FAU ", "FEA ou FAU II", "As vezes desembarco na FAU II ou na FEA"]:
        return "Ponto de ônibus FAU II"
    elif x == "Portaria São Remo":
        return "Ponto de ônibus acesso Rio Pequeno"
    elif x == "If e Ipen":
        return "Ponto de ônibus Física"
    elif x == "Casa de Cultura Japonesa, ETEC CEPAM":
        return "Ponto de ônibus Cultura Japonesa"
    elif x == "IEB":
        return "Ponto de ônibus Biblioteca Brasiliana"
    elif x in ["IAG (ou, às vezes, Física)", "IAG ou Física", "IAG (sentido Metrô Butantã)"]:
        return "Ponto de ônibus IAG"
    elif x == "Prof Almeida Prado":
        return "Terminal de ônibus USP"
    elif x == "HU":
        return "Ponto de ônibus da COPESP"
    elif x == "pço das artes":
        return "Ponto de ônibus Paços das Artes"
    elif x == "Depende do dia":
        return None
    else:
        return x

In [871]:
pesquisa_vinculo_intervalo_butanta['ponto_' + acao + '_' + sentido] = list(map(lambda x: pesquisa_to_pontos_req(iguais_com_prefixo(x)), pesquisa_vinculo_intervalo_butanta['ponto_' + acao + '_' + sentido]))
pesquisa_vinculo_intervalo_butanta = pesquisa_vinculo_intervalo_butanta[pesquisa_vinculo_intervalo_butanta['ponto_' + acao + '_' + sentido].notna()]
pesquisa_vinculo_intervalo_butanta['ponto_' + acao + '_' + sentido].unique()

for ponto in pesquisa_vinculo_intervalo_butanta['ponto_' + acao + '_' + sentido].unique():
    if ponto not in nomes_pontos_req:
        print(ponto)

IEE
IEE - Instituto de Energia e Ambiente
Início da rua do Matão, próximo ao ICB
Reitoria - Antiga, Ponto dos Restaurantes 


##### Calcular Porcentagem Carro por Instituto

In [872]:
porc_circular_por_instituto = pesquisa_vinculo_intervalo_butanta[['instituto','utiliza_circular_' + sentido]].groupby('instituto').agg('mean').reset_index()
porc_circular_por_instituto

,instituto,utiliza_circular_ida
0,Escola Politécnica,1.0
1,Faculdade de Ciências Farmacêuticas,1.0
2,"Faculdade de Economia, Administração, Contabil...",1.0
3,"Faculdade de Filosofia, Letras e Ciências Humanas",1.0
4,Faculdade de Medicina Veterinária e Zootecnia,1.0
5,"Faculdade de Medicina, Instituto de Ciências B...",1.0
6,Faculdade de Odontologia,1.0
7,Hospital Universitario,1.0
8,Hospital Universitário,1.0
9,Instituto Oceanográfico,1.0


In [873]:
def porc_utiliza_circular(instituto):
    return porc_circular_por_instituto[porc_circular_por_instituto['instituto'] == instituto]['utiliza_circular_' + sentido].iloc[0]
porc_utiliza_circular('Instituto de Física')

1.0

##### Calcula porcentagem para Ponto e Linha por Instituto (filtrando quem usa carro)

In [874]:
import random
def usa_linha(str_linhas, linha_target):
    linhas = str_linhas.split(', ')
    linha_selecionada = linhas[random.randint(0, len(linhas) - 1)]
    return (linha_selecionada == linha_target)

def calcula_quantidade_pessoas_ponto(instituto, ponto, linha):
    df_instituto = pesquisa_vinculo_intervalo_butanta[pesquisa_vinculo_intervalo_butanta['instituto'] == instituto]

    df_linha = df_instituto[list(map(lambda x: usa_linha(x, linha), df_instituto['linhas_' + sentido]))]    
    if len(df_linha['linhas_' + sentido]) == 0:
        return 0

    df_ponto = df_linha[df_linha['ponto_' + acao + '_' + sentido] == ponto]
    if len(df_ponto['ponto_' + acao + '_' + sentido]) == 0:
        return 0

    return df_ponto['ponto_' + acao + '_' + sentido].value_counts().iloc[0]

In [875]:
pontos_existentes = pesquisa_vinculo_intervalo_butanta['ponto_' + acao + '_' + sentido].unique()

institutos_ponto_rel = {}
for instituto in pesquisa_vinculo_intervalo_butanta['instituto'].unique():
    institutos_ponto_rel[instituto] = {}
    soma = len(pesquisa_vinculo_intervalo_butanta[pesquisa_vinculo_intervalo_butanta['instituto'] == instituto]['instituto'])
    for ponto in pontos_existentes:
        for linha in ['8012', '8022', '8032']:
            pessoas = calcula_quantidade_pessoas_ponto(instituto, ponto, linha) * porc_utiliza_circular(instituto)
            if pessoas > 0:
                if ponto not in institutos_ponto_rel[instituto]:
                    institutos_ponto_rel[instituto][ponto] = {}
                institutos_ponto_rel[instituto][ponto][linha] = pessoas / soma
institutos_ponto_rel

{'Hospital Universitário': {'Ponto de ônibus Hospital Universitário': {'8012': 0.5714285714285714,
   '8022': 0.19047619047619047,
   '8032': 0.047619047619047616},
  'Ponto de ônibus acesso Rio Pequeno': {'8012': 0.047619047619047616}},
 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências': {'Ponto de ônibus Hospital Universitário': {'8012': 0.25,
   '8022': 0.25},
  'Ponto de ônibus Reitoria/Bancos': {'8012': 0.25},
  'Ponto de ônibus Portaria III': {'8022': 0.25}},
 'Instituto de Ciências Biomédicas': {'Ponto de ônibus Biomédicas III': {'8012': 0.2,
   '8022': 0.4},
  'Ponto de ônibus acesso Vl. Indiana': {'8022': 0.2}},
 'Reitoria Administração Central': {'Ponto de ônibus Cocesp I': {'8022': 0.08333333333333333},
  'Ponto de ônibus Reitoria/Bancos': {'8012': 0.08333333333333333,
   '8032': 0.08333333333333333},
  'Ponto de ônibus CRUSP': {'8022': 0.25, '8032': 0.08333333333333333},
  'Ponto de ônibus Biblioteca Brasiliana': {'80

# Pesquisa e (Aluno/DadosMatrusp ou Funcionário)

In [876]:
aulas_intervalo_horario = get_dados_vinculo(aulas_intervalo_horario, funcionarios_intervalo_horario)

##### Verificar unidades Matrusp e Pesquisa

In [877]:
# Alterar a unidade de aulas na qual a unidade não respondeu a pesquisa

for aula in aulas_intervalo_horario:
    if aula['unidade'] == 'Instituto de Ciências Biomédicas':
        aula['unidade'] = 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências'
    if aula['unidade'] == 'Instituto de Estudos Brasileiros':
        aula['unidade'] = 'Faculdade de Economia, Administração, Contabilidade e Atuária'
    if aula['unidade'] == 'Instituto de Psicologia':
        aula['unidade'] = 'Escola de Comunicações e Artes'

In [878]:
institutos_matrusp = {}
for aula in aulas_intervalo_horario:
    institutos_matrusp[aula['unidade']] = 1
institutos_matrusp = list(institutos_matrusp.keys())
institutos_matrusp.sort()
institutos_matrusp

['Centro de Práticas Esportivas',
 'Escola Politécnica',
 'Escola de Comunicações e Artes',
 'Faculdade de Arquitetura e Urbanismo',
 'Faculdade de Ciências Farmacêuticas',
 'Faculdade de Economia, Administração, Contabilidade e Atuária',
 'Faculdade de Educação',
 'Faculdade de Filosofia, Letras e Ciências Humanas',
 'Faculdade de Medicina Veterinária e Zootecnia',
 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências',
 'Faculdade de Odontologia',
 'Hospital Universitário',
 'Instituto Oceanográfico',
 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas',
 'Instituto de Biociências',
 'Instituto de Física',
 'Instituto de Geociências',
 'Instituto de Matemática e Estatística',
 'Instituto de Pesquisas Energéticas e Nucleares',
 'Instituto de Química',
 'Instituto de Relações Internacionais',
 'Museu de Arte Contemporânea',
 'Reitoria Administração Central',
 'Superintendência da Tecnologia da Informação']

In [879]:
institutos_pesquisa = list(institutos_ponto_rel.keys())
institutos_pesquisa.sort()
institutos_pesquisa

['Escola Politécnica',
 'Faculdade de Ciências Farmacêuticas',
 'Faculdade de Economia, Administração, Contabilidade e Atuária',
 'Faculdade de Filosofia, Letras e Ciências Humanas',
 'Faculdade de Medicina Veterinária e Zootecnia',
 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências',
 'Faculdade de Odontologia',
 'Hospital Universitario',
 'Hospital Universitário',
 'Instituto Oceanográfico',
 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas',
 'Instituto de Biociências',
 'Instituto de Ciências Biomédicas',
 'Instituto de Energia e Ambiente',
 'Instituto de Estudos Avançados',
 'Instituto de Física',
 'Instituto de Geociências',
 'Instituto de Matemática e Estatística',
 'Instituto de Psicologia',
 'Instituto de Química',
 'Instituto de Relações Internacionais',
 'Reitoria Administração Central']

In [880]:
for instituto_matrusp in institutos_matrusp:
    if instituto_matrusp not in institutos_pesquisa:
        print(instituto_matrusp + ' from matrusp is not in institutos pesquisa')

Centro de Práticas Esportivas from matrusp is not in institutos pesquisa
Escola de Comunicações e Artes from matrusp is not in institutos pesquisa
Faculdade de Arquitetura e Urbanismo from matrusp is not in institutos pesquisa
Faculdade de Educação from matrusp is not in institutos pesquisa
Instituto de Pesquisas Energéticas e Nucleares from matrusp is not in institutos pesquisa
Museu de Arte Contemporânea from matrusp is not in institutos pesquisa
Superintendência da Tecnologia da Informação from matrusp is not in institutos pesquisa


In [881]:
institutos_ponto_rel['Escola Politécnica']

{'Ponto de ônibus Poli Biênio': {'8022': 0.125, '8032': 0.25},
 'Ponto de ônibus metrô Butantã': {'8022': 0.125},
 'Ponto de ônibus Poli Civil': {'8022': 0.125},
 'Ponto de ônibus Butantan': {'8012': 0.125},
 'Ponto de ônibus Poli Eletrotécnica': {'8032': 0.125}}

##### Calcula Map: dia->hora->ponto->linha->pessoas

In [882]:
dia_hora_ponto_linha = {}

def sum_valor_dia_hora_ponto_linha(dia, hora, ponto, linha, valor):
    if dia not in dia_hora_ponto_linha:
        dia_hora_ponto_linha[dia] = {}
    if hora not in dia_hora_ponto_linha[dia]:
        dia_hora_ponto_linha[dia][hora] = {}
    if ponto not in dia_hora_ponto_linha[dia][hora]:
        dia_hora_ponto_linha[dia][hora][ponto] = {}
    if linha not in dia_hora_ponto_linha[dia][hora][ponto]:
        dia_hora_ponto_linha[dia][hora][ponto][linha] = 0
    dia_hora_ponto_linha[dia][hora][ponto][linha] += valor

for aula in aulas_intervalo_horario:
    if aula['unidade'] in institutos_ponto_rel:
        porcentagem_pontos_por_instituto = institutos_ponto_rel[aula['unidade']]
        for ponto in porcentagem_pontos_por_instituto:
            linhas = porcentagem_pontos_por_instituto[ponto]
            for linha in linhas:
                porc = linhas[linha]
                valor = aula['total'] * porc
                sum_valor_dia_hora_ponto_linha(aula['dia'], hora, ponto, linha, valor)

# arredondando
for dia in dia_hora_ponto_linha:
    for hora in dia_hora_ponto_linha[dia]:
        for ponto in dia_hora_ponto_linha[dia][hora]:
            for linha in dia_hora_ponto_linha[dia][hora][ponto]:
                dia_hora_ponto_linha[dia][hora][ponto][linha] = round(dia_hora_ponto_linha[dia][hora][ponto][linha])

dia_hora_ponto_linha

{'seg': {1080: {'Ponto de ônibus Física': {'8012': 14},
   'Ponto de ônibus FEA': {'8032': 13, '8022': 12},
   'Ponto de ônibus Reitoria/Bancos': {'8032': 52, '8012': 52},
   'Ponto de ônibus Letras': {'8012': 15, '8032': 15},
   'Ponto de ônibus Biblioteca Brasiliana': {'8012': 15, '8032': 37},
   'Ponto de ônibus Portaria III': {'8012': 20, '8022': 35},
   'Ponto de ônibus Biblioteca Farmácia e Química': {'8012': 15, '8022': 56},
   'Ponto de ônibus Poli Biênio': {'8022': 30, '8032': 45, '8012': 11},
   'Ponto de ônibus Odontologia': {'8012': 14},
   'Ponto de ônibus metrô Butantã': {'8022': 26},
   'Ponto de ônibus Poli Civil': {'8022': 18},
   'Ponto de ônibus Butantan': {'8012': 18},
   'Ponto de ônibus Poli Eletrotécnica': {'8032': 18},
   'Ponto de ônibus CEPAM': {'8012': 15, '8022': 15},
   'Ponto de ônibus  Parada Matemática': {'8012': 15, '8022': 29},
   'Início da rua do Matão, próximo ao ICB': {'8022': 15},
   'Ponto de ônibus - Acesso CPTM I': {'8022': 9},
   'Ponto de ôni

##### Salva Map em Arquivo

In [883]:
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

def encode(s):
    return s.encode('latin1').decode('unicode_escape')

with open(map_file_name, 'w+') as f:
    str_dia_hora_ponto_linha = encode(json.dumps(dia_hora_ponto_linha, indent=4, cls=NpEncoder))
    f.write(str_dia_hora_ponto_linha)
    print(str_dia_hora_ponto_linha)

{
    "seg": {
        "1080": {
            "Ponto de ônibus Física": {
                "8012": 14
            },
            "Ponto de ônibus FEA": {
                "8032": 13,
                "8022": 12
            },
            "Ponto de ônibus Reitoria/Bancos": {
                "8032": 52,
                "8012": 52
            },
            "Ponto de ônibus Letras": {
                "8012": 15,
                "8032": 15
            },
            "Ponto de ônibus Biblioteca Brasiliana": {
                "8012": 15,
                "8032": 37
            },
            "Ponto de ônibus Portaria III": {
                "8012": 20,
                "8022": 35
            },
            "Ponto de ônibus Biblioteca Farmácia e Química": {
                "8012": 15,
                "8022": 56
            },
            "Ponto de ônibus Poli Biênio": {
                "8022": 30,
                "8032": 45,
                "8012": 11
            },
            "Ponto de ônibus Odo

# Outros

## Add employees and professors
TODO: considerar funcionários e professores estrapolando o horário de entrada da pesquisa

##### Número Pessoas por Dia e Instituto

In [884]:
agrupamento_dia_instituto = {}
for dia in dias:
    agrupamento_dia_instituto[dia] = {}
for aula in aulas_intervalo_horario:
    if aula['unidade'] not in agrupamento_dia_instituto[aula['dia']]:
        agrupamento_dia_instituto[aula['dia']][aula['unidade']] = 0
    agrupamento_dia_instituto[aula['dia']][aula['unidade']] += aula['total']
agrupamento_dia_instituto

{'seg': {'Faculdade de Arquitetura e Urbanismo': 9.923076923076923,
  'Instituto de Física': 27.692307692307693,
  'Faculdade de Economia, Administração, Contabilidade e Atuária': 26.25,
  'Faculdade de Filosofia, Letras e Ciências Humanas': 77.08695652173914,
  'Museu de Arte Contemporânea': 12.0,
  'Faculdade de Medicina Veterinária e Zootecnia': 59.047619047619044,
  'Faculdade de Ciências Farmacêuticas': 44.666666666666664,
  'Instituto de Matemática e Estatística': 24.5,
  'Escola de Comunicações e Artes': 59.79761904761904,
  'Instituto de Pesquisas Energéticas e Nucleares': 0.0,
  'Faculdade de Odontologia': 13.833333333333332,
  'Escola Politécnica': 141.05882352941177,
  'Instituto de Relações Internacionais': 9.666666666666666,
  'Faculdade de Educação': 9.176470588235293,
  'Instituto de Biociências': 58.58823529411765,
  'Instituto de Química': 43.63333333333333,
  'Instituto de Geociências': 18.666666666666664,
  'Instituto de Astronomia, Geofísica e Ciências Atmosféricas'

In [885]:
for dia in agrupamento_dia_instituto:
    for instituto in agrupamento_dia_instituto[dia]:
        if len(porc_circular_por_instituto[porc_circular_por_instituto['instituto'] == instituto]['utiliza_circular_ida']) > 0:
            agrupamento_dia_instituto[dia][instituto] *= porc_circular_por_instituto[porc_circular_por_instituto['instituto'] == instituto]['utiliza_circular_ida'].iloc[0]
        else:
            agrupamento_dia_instituto[dia][instituto] = 0
        agrupamento_dia_instituto[dia][instituto] = int(agrupamento_dia_instituto[dia][instituto])

# Pessoas por instituto que utilizam circular de acordo com a pesquisa
agrupamento_dia_instituto

{'seg': {'Faculdade de Arquitetura e Urbanismo': 0,
  'Instituto de Física': 27,
  'Faculdade de Economia, Administração, Contabilidade e Atuária': 26,
  'Faculdade de Filosofia, Letras e Ciências Humanas': 77,
  'Museu de Arte Contemporânea': 0,
  'Faculdade de Medicina Veterinária e Zootecnia': 59,
  'Faculdade de Ciências Farmacêuticas': 44,
  'Instituto de Matemática e Estatística': 24,
  'Escola de Comunicações e Artes': 0,
  'Instituto de Pesquisas Energéticas e Nucleares': 0,
  'Faculdade de Odontologia': 13,
  'Escola Politécnica': 141,
  'Instituto de Relações Internacionais': 9,
  'Faculdade de Educação': 0,
  'Instituto de Biociências': 58,
  'Instituto de Química': 43,
  'Instituto de Geociências': 18,
  'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 10,
  'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências': 63,
  'Hospital Universitário': 607,
  'Instituto Oceanográfico': 33,
  'Reitoria Administração Ce